In [1]:
import pandas as pd
ratings = pd.read_csv('data/ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

In [2]:
import pandas as pd

data = pd.read_csv('data/ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
data.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
info = pd.read_csv('data/ml-100k/u.info', sep='\t', names=['key', 'value'])
info.head()

,key,value
0,943 users,NaN
1,1682 items,NaN
2,100000 ratings,NaN


In [4]:
item = pd.read_csv('data/ml-100k/u.item', sep='|', encoding='latin-1', names=['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL'] + [f'genre_{i}' for i in range(19)])
item.head()

,movie_id,title,release_date,video_release_date,IMDb_URL,genre_0,genre_1,genre_2,genre_3,genre_4,...,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
genres = pd.read_csv('data/ml-100k/u.genre', sep='|', names=['genre', 'genre_id'])
genres.head()

,genre,genre_id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


In [6]:
user = pd.read_csv('data/ml-100k/u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])
user.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [7]:
# data : 레이팅 정보
# Info : 영화 정보 (장르, 출시년도)
# user : 유저 정보 (나이 성별 직업)
# 여기서 어떻게 할 수 잇을까? -> 이걸 어떻게 변수로 쓸 수 잇는가?

data_user = pd.merge(data, user, on='user_id')
data_user.head()

,user_id,movie_id,rating,timestamp,age,gender,occupation,zip_code
0,196,242,3,881250949,49,M,writer,55105
1,186,302,3,891717742,39,F,executive,00000
2,22,377,1,878887116,25,M,writer,40206
3,244,51,2,880606923,28,M,technician,80525
4,166,346,1,886397596,47,M,educator,55113


In [8]:
data_user_item = pd.merge(data_user, item, on='movie_id')
data_user_item.head()

,user_id,movie_id,rating,timestamp,age,gender,occupation,zip_code,title,release_date,...,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
0,196,242,3,881250949,49,M,writer,55105,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,39,F,executive,00000,L.A. Confidential (1997),01-Jan-1997,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,25,M,writer,40206,Heavyweights (1994),01-Jan-1994,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,28,M,technician,80525,Legends of the Fall (1994),01-Jan-1994,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,47,M,educator,55113,Jackie Brown (1997),01-Jan-1997,...,0,0,0,0,0,0,0,0,0,0


In [9]:
import pandas as pd
import numpy as np

# 1. 나이 범주화 (Age Binning)
# 0-29(1020대), 30-49(3040대), 50-69(5060대), 70+(그외)
bins_age = [0, 29, 49, 69, 120]
labels_age = ['1020s', '3040s', '5060s', 'others']
data_user_item['age_group'] = pd.cut(data_user_item['age'], bins=bins_age, labels=labels_age)

# 3. 원-핫 인코딩 (One-Hot Encoding)
# age_group과 year_group을 실제 모델에 넣을 0과 1의 컬럼들로 변환합니다.
data_final = pd.get_dummies(data_user_item, columns=['age_group'], prefix=['age'])

# 확인
print(data_final.columns)
data_final.head()

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'age', 'gender',
       'occupation', 'zip_code', 'title', 'release_date', 'video_release_date',
       'IMDb_URL', 'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4',
       'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10',
       'genre_11', 'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16',
       'genre_17', 'genre_18', 'age_1020s', 'age_3040s', 'age_5060s',
       'age_others'],
      dtype='object')


,user_id,movie_id,rating,timestamp,age,gender,occupation,zip_code,title,release_date,...,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18,age_1020s,age_3040s,age_5060s,age_others
0,196,242,3,881250949,49,M,writer,55105,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,False,True,False,False
1,186,302,3,891717742,39,F,executive,00000,L.A. Confidential (1997),01-Jan-1997,...,1,0,0,1,0,0,False,True,False,False
2,22,377,1,878887116,25,M,writer,40206,Heavyweights (1994),01-Jan-1994,...,0,0,0,0,0,0,True,False,False,False
3,244,51,2,880606923,28,M,technician,80525,Legends of the Fall (1994),01-Jan-1994,...,0,1,0,0,1,1,True,False,False,False
4,166,346,1,886397596,47,M,educator,55113,Jackie Brown (1997),01-Jan-1997,...,0,0,0,0,0,0,False,True,False,False


In [10]:
drop_cols = [
    'timestamp', 'age', 'zip_code', 'title', 'release_date', 
    'video_release_date', 'IMDb_URL','occupation','gender'
]

# 컬럼 삭제 (inplace=True를 쓰면 원본이 바로 수정됩니다)
data_final.drop(columns=drop_cols, inplace=True, errors='ignore')

# 결과 확인
print("✅ 정리된 컬럼 목록:")
print(data_final.columns)
data_final.dtypes

✅ 정리된 컬럼 목록:
Index(['user_id', 'movie_id', 'rating', 'genre_0', 'genre_1', 'genre_2',
       'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7', 'genre_8',
       'genre_9', 'genre_10', 'genre_11', 'genre_12', 'genre_13', 'genre_14',
       'genre_15', 'genre_16', 'genre_17', 'genre_18', 'age_1020s',
       'age_3040s', 'age_5060s', 'age_others'],
      dtype='object')


user_id       int64
movie_id      int64
rating        int64
genre_0       int64
genre_1       int64
genre_2       int64
genre_3       int64
genre_4       int64
genre_5       int64
genre_6       int64
genre_7       int64
genre_8       int64
genre_9       int64
genre_10      int64
genre_11      int64
genre_12      int64
genre_13      int64
genre_14      int64
genre_15      int64
genre_16      int64
genre_17      int64
genre_18      int64
age_1020s      bool
age_3040s      bool
age_5060s      bool
age_others     bool
dtype: object

In [11]:
# 3. 만약 bool 타입이 여전히 남아있다면 강제로 float로 변환 (파이토치 호환용)
bool_cols = data_final.select_dtypes(include=['bool']).columns
data_final[bool_cols] = data_final[bool_cols].astype(float)

# 확인
print("✅ 인코딩 후 컬럼 목록:")
print(data_final.columns)
print("\n📊 현재 데이터 타입 확인:")
print(data_final.dtypes.value_counts()) # object가 0개여야 합니다!

✅ 인코딩 후 컬럼 목록:
Index(['user_id', 'movie_id', 'rating', 'genre_0', 'genre_1', 'genre_2',
       'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7', 'genre_8',
       'genre_9', 'genre_10', 'genre_11', 'genre_12', 'genre_13', 'genre_14',
       'genre_15', 'genre_16', 'genre_17', 'genre_18', 'age_1020s',
       'age_3040s', 'age_5060s', 'age_others'],
      dtype='object')

📊 현재 데이터 타입 확인:
int64      22
float64     4
Name: count, dtype: int64


In [12]:
data_final.to_csv('data/processed_data_v2.csv', index=False)